In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [2]:
vocab_size = 1500
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

1909
12063
[7, 163, 68, 81, 396, 21, 14, 44, 62, 78]
[163, 68, 81, 396, 21, 14, 44, 62, 78, 10]


In [21]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [22]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [23]:
# def sam(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [13]:
# hyper-parameters
epochs = 10000
hidden_layer = 128
inp_out_size = 1500
learning_rate = 0.1
num_steps = 15

In [6]:
tf.reset_default_graph()
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.random_normal([hidden_layer, inp_out_size], mean=0, stddev=0.001))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
print tf.trainable_variables()

[<tensorflow.python.ops.variables.Variable object at 0x7f8b8a08d2d0>, <tensorflow.python.ops.variables.Variable object at 0x7f8b89fe4850>, <tensorflow.python.ops.variables.Variable object at 0x7f8b89f5c450>, <tensorflow.python.ops.variables.Variable object at 0x7f8b8a00aa90>, <tensorflow.python.ops.variables.Variable object at 0x7f8b8a00aad0>]


In [7]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [8]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [16]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [10]:
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
ix = 0
smooth_loss = -np.log(1.0 / vocab_size)*num_steps

In [11]:
# if already trained previously, just restore
to_restore = True

if to_restore:
    saver.restore(sess, 'model.ckpt')
else:
    sess.run(init)

In [14]:
print epochs
h = np.zeros(hidden_layer)
for i in range(epochs):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
    num_steps = num_steps - 1 if i % 2 == 0 else num_steps + 1 # variable length sequences
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, h, _ = sess.run([loss, states, optimize_op], {a: x, b: y, initial: h}) 
    smooth_loss = smooth_loss * 0.999 + l * 0.001
    if i % 1000 == 0:
        print 'epoch {0}, loss = {1}'.format(i, smooth_loss)
#         print sess.run(Whh)[0][:30]
    ix += num_steps
        
#         generate(sess,40)

10000
epoch 0, loss = 15.9441945378
epoch 1000, loss = 12.7052840564
epoch 2000, loss = 11.2963803434
epoch 3000, loss = 10.468597103
epoch 4000, loss = 10.0458583511
epoch 5000, loss = 9.87956408773
epoch 6000, loss = 9.83136852966
epoch 7000, loss = 9.53692478479
epoch 8000, loss = 9.21046329804
epoch 9000, loss = 9.04049821997


In [18]:
# saving model
saver.save(sess, 'model.ckpt')

'model.ckpt'

In [17]:
generate(sess, 500)

terrorists and I said , we are n't unk unk the place . They are n't them paying the most place . 
 
 The country ; you have a big where they like this , I 'm going to say but other countries , unk do not going to unk our country . unk you 've him that look at how to have a very important election . 
 
 You know I said oh we 're just — and the deal . I know many of it all over the way , but this country ; we got it . And you are n't the other day we can see so many of that and we do n't know what we are doing . They 're building the one that statement ; that got some heard of our country . And you ca n't have unk that but what are doing ? I like the other day . I mean you all the unk election in -- it 's something to unk . It 's n't unk to them . It was just them in Iraq . So we have been selling this other effort ago , I 'm a unk election coming up what can know and oh talk come in unk unk airport and you can do n't us ; you are some of them . 
 
 The country 's not . So they do the ri

In [33]:
sess.run(Whh)

array([[ 0.03073823,  0.16024236,  0.16885728, ...,  0.05288818,
        -0.07688183, -0.03028169],
       [-0.24785835, -0.04390815,  0.1450949 , ..., -0.05311445,
        -0.01250314,  0.11281139],
       [-0.33347312, -0.20284222,  0.23098749, ..., -0.10504574,
        -0.28402022,  0.03956333],
       ..., 
       [ 0.05662825,  0.16807128,  0.00710124, ...,  0.22454964,
        -0.06467728,  0.10287257],
       [-0.06466194, -0.10157995, -0.11470366, ...,  0.09235065,
         0.3563292 ,  0.02229096],
       [ 0.08362083,  0.13572359,  0.1389745 , ...,  0.22135787,
         0.00691832,  0.11428044]], dtype=float32)